In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
WORK_DIR = os.path.join('https://drive.google.com/drive/u/0/my-drive')

## Выбираем 10000+ изображений

In [3]:
def txt2csv(txt, csv, drop_first_line=True, colname = True):        
    with open(txt, 'r') as f_txt:
        with open(csv, 'w') as f_csv:
            if drop_first_line:
                f_txt.readline()
            if not colname:
                f_txt.readline()
            while (line := f_txt.readline()) != '':
                old_line = line
                line = line.replace('  ', ' ')
                while (old_line != line):
                    old_line = line
                    line = line.replace('  ', ' ')
                f_csv.write(line.replace(' ', ','))


Текстовые файлы переводятся в формат csv

In [ ]:
txt2csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_landmarks_celeba.txt'),
        os.path.join(WORK_DIR,'celeba-original-wild-images', 'list_landmarks_celeba_new.csv'), colname=False)
txt2csv(os.path.join(WORK_DIR,'celeba-original-wild-images', 'identity_CelebA.txt'),
        os.path.join(WORK_DIR,'celeba-original-wild-images', 'identity_CelebA.csv'), drop_first_line=False)

In [ ]:
attr = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_attr_celeba.csv'))
bbox = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_bbox_celeba.csv'))
identity = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'identity_CelebA.csv'), names=['image_id', 'id'],
                        na_values=['', 'NULL', 'N/A'])
landmarks = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_landmarks_celeba_new.csv'))

In [6]:
columns = attr.columns.drop('image_id').to_list()
for i, column in tqdm(enumerate(columns, 1)):
    if i % 5 != 0:
        print(f'{column}', ' '*(19 - len(column)), end = '', sep= ' ')
    else:
        print(f'{column}', ' '*(19 - len(column)), sep= ' ')

40it [00:00, 17313.95it/s]

5_o_Clock_Shadow    Arched_Eyebrows     Attractive          Bags_Under_Eyes     Bald                
Bangs               Big_Lips            Big_Nose            Black_Hair          Blond_Hair          
Blurry              Brown_Hair          Bushy_Eyebrows      Chubby              Double_Chin         
Eyeglasses          Goatee              Gray_Hair           Heavy_Makeup        High_Cheekbones     
Male                Mouth_Slightly_Open Mustache            Narrow_Eyes         No_Beard            
Oval_Face           Pale_Skin           Pointy_Nose         Receding_Hairline   Rosy_Cheeks         
Sideburns           Smiling             Straight_Hair       Wavy_Hair           Wearing_Earrings    
Wearing_Hat         Wearing_Lipstick    Wearing_Necklace    Wearing_Necktie     Young               


#### Описание list_attr_celeba.csv

image_id      
5_o_Clock_Shadow     -  щетина на лице мужчины      
Arched_Eyebrows      -  изогнутые брови      
Attractive           -  привлекательный           
Bags_Under_Eyes      -  мешки под глазами      
Bald                 -  лысый       
Bangs                -  волосы, подстриженные на передней части головы и обрамляющие лицо (челка)      
Big_Lips             -  большие губы     
Big_Nose             -  большой нос     
Black_Hair           -  черные волосы       
Blond_Hair           -  светлые волосы     
Blurry               -  размытый       
Brown_Hair           -  каштановые волосы       
Bushy_Eyebrows       -  густые брови      
Chubby               -  круглолицый      
Double_Chin          -  двойной подбородок       
Eyeglasses           -  очки       
Goatee               -  козлиная бородка       
Gray_Hair            -  седые волосы       
Heavy_Makeup         -  тяжелый макияж      
High_Cheekbones      -  высокие скулы      
Male                 -  мужчина      
Mouth_Slightly_Open  -  слегка приоткрытый рот          
Mustache             -  усы            
Narrow_Eyes          -  узкие глаза                 
No_Beard             -  без бороды                     
Oval_Face            -  овальное лицо               
Pale_Skin            -  бледная кожа           
Pointy_Nose          -  заостренный нос          
Receding_Hairline    -  редеющая линия волос          
Rosy_Cheeks          -  румяные щеки            
Sideburns            -  бакенбарды                
Smiling              -  улыбающийся          
Straight_Hair        -  прямые волосы             
Wavy_Hair            -  волнистые волосы             
Wearing_Earrings     -  носит серьги           
Wearing_Hat          -  носит шляпу           
Wearing_Lipstick     -  использует губную помаду          
Wearing_Necklace     -  носит шнурок на шее           
Wearing_Necktie      -  носит галстук        
Young                -  молодой           

#### Создаем папку для выборки и преносим в нее выбранные файлы.

Удаляем из списка файлы с баудингбоксами x_1 < 0, y_1 < 0, width <= 0 и height <= 0  

In [7]:
print(len(attr))
drop_files = bbox[(bbox['x_1'] < 0) | (bbox['y_1'] < 0) | (bbox['width'] <= 0) | (bbox['height'] <= 0)]
attr = attr[~attr.image_id.isin(drop_files.image_id)]
print(drop_files)
print(len(attr))

202599
          image_id  x_1  y_1  width  height
101282  101283.jpg  320  828      0       0
202598


##### Отбираем людей с количеством фото равным n_photos

In [8]:
n_photos = 30

In [9]:
selected_futures = identity.groupby('id').filter(lambda x: len(x) == n_photos)
len(selected_futures)

70290

In [10]:
select_10000 = attr[attr.image_id.isin(selected_futures.image_id)]
len(select_10000)

70290

In [ ]:
os.makedirs(os.path.join(WORK_DIR, '10000'), exist_ok=True)

In [ ]:
select_10000.to_csv(os.path.join(WORK_DIR, '10000', '10000.csv'), index=False)

In [13]:
select_10000.sort_values(by='image_id').head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
5,000006.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,1,-1,1,-1,-1,1


##### Копируем выбранные файлы в отдельный каталог

In [ ]:
import shutil
dir_files = {}
dirnames = os.scandir(os.path.join(WORK_DIR, 'celeba-original-wild-images'))
dirnames = [dir.name for dir in dirnames if dir.is_dir()]
for dirname in dirnames:
    for filename in os.listdir(os.path.join(WORK_DIR, 'celeba-original-wild-images', dirname, dirname)):
        full_path =os.path.join(WORK_DIR, 'celeba-original-wild-images', dirname, dirname, filename)
        dir_files[filename] = full_path
list_files = pd.DataFrame({'image_id' : dir_files.keys(), 'full_path' : dir_files.values()})
files_10000 = list_files[list_files.image_id.isin(select_10000.image_id)]
print(len(files_10000))
for idx, filename in tqdm(files_10000.iterrows()):
    result = shutil.copy2(filename['full_path'], os.path.join(WORK_DIR, '10000', filename['image_id']))


70290


70290it [37:42, 31.06it/s] 


In [15]:
files_10000.head()

,image_id,full_path
0,000001.jpg,C:\Users\zabro\OneDrive\Рабочий стол\Face Reco...
1,000002.jpg,C:\Users\zabro\OneDrive\Рабочий стол\Face Reco...
2,000003.jpg,C:\Users\zabro\OneDrive\Рабочий стол\Face Reco...
3,000004.jpg,C:\Users\zabro\OneDrive\Рабочий стол\Face Reco...
5,000006.jpg,C:\Users\zabro\OneDrive\Рабочий стол\Face Reco...


In [ ]:
copy_10000 = pd.DataFrame({'image_id' :list(os.listdir(os.path.join(WORK_DIR, '10000')))})
copy_10000.head()

,image_id
0,000001.jpg
1,000002.jpg
2,000003.jpg
3,000004.jpg
4,000006.jpg


Проверка все ли выбранные файлы скопированы.

In [17]:
copy_10000[~copy_10000.image_id.isin(files_10000.image_id)]

,image_id
34388,10000.csv


In [18]:
copy_10000[~copy_10000.image_id.isin(select_10000.image_id)]

,image_id
34388,10000.csv


### Обрезаем изображения и оставляем только лица

#### Выбираем баудингбоксы для выбранных файлов и записываем их в csv-файл. 

In [ ]:
select_10000 = pd.read_csv(os.path.join(WORK_DIR, '10000', '10000.csv'))
bbox = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_bbox_celeba.csv'))
bbox_10000 = bbox[bbox.image_id.isin(select_10000.image_id)]

In [ ]:
bbox_10000.to_csv(os.path.join(WORK_DIR, '10000', 'bbox_10000.csv'), index=False)

In [ ]:
bbox_10000 = pd.read_csv(os.path.join(WORK_DIR, '10000', 'bbox_10000.csv'))

In [22]:
bbox_10000.head()

,image_id,x_1,y_1,width,height
0,000001.jpg,95,71,226,313
1,000002.jpg,72,94,221,306
2,000003.jpg,216,59,91,126
3,000004.jpg,622,257,564,781
4,000006.jpg,146,67,182,252


#### Вырезаем лица из изображений и помещаем их отдельный каталог 

In [ ]:
# каталог для лиц
os.makedirs(os.path.join(WORK_DIR, '10000', 'faces'), exist_ok=True)

In [ ]:
from PIL import Image               
import torch
import torchvision.transforms as transforms
for index, row in tqdm(bbox_10000.iterrows()):
    pil_im = Image.open(os.path.join(WORK_DIR, '10000', row['image_id']))
    crop = transforms.functional.crop(pil_im,
                                       row['y_1'], row['x_1'], row['height'], row['width'])
    crop.save(os.path.join(WORK_DIR, '10000', 'faces', row['image_id']))

70290it [1:10:06, 16.71it/s]


### Разметка для выбранных 10000+ лиц 

Изменяем координаты глаз, носа, рта исходя из новых размеров изображений 

In [ ]:
select_10000 = pd.read_csv(os.path.join(WORK_DIR, '10000', '10000.csv'))
landmarks = pd.read_csv(os.path.join(WORK_DIR, 'celeba-original-wild-images', 'list_landmarks_celeba_new.csv'), names =
        {'image_id' : object, 'lefteye_x' : int, 'lefteye_y' : int, 'righteye_x' : int, 'righteye_y' : int,
         'nose_x' : int, 'nose_y' : int, 'leftmouth_x' : int, 'leftmouth_y' : int, 'rightmouth_x' : int, 'rightmouth_y' : int})
landmarks_10000 = landmarks[landmarks.image_id.isin(select_10000.image_id)]
merged = pd.merge(landmarks_10000, bbox_10000, on= 'image_id')
merged.lefteye_x = merged.lefteye_x - merged.x_1
merged.righteye_x = merged.righteye_x - merged.x_1
merged.nose_x = merged.nose_x - merged.x_1
merged.leftmouth_x = merged.leftmouth_x - merged.x_1
merged.rightmouth_x = merged.rightmouth_x - merged.x_1

merged.lefteye_y = merged.lefteye_y - merged.y_1
merged.righteye_y = merged.righteye_y - merged.y_1
merged.nose_y = merged.nose_y - merged.y_1
merged.leftmouth_y = merged.leftmouth_y - merged.y_1
merged.rightmouth_y = merged.rightmouth_y - merged.y_1
landmarks_10000 = merged.drop(columns=['x_1', 'y_1', 'width', 'height'])
landmarks_10000.to_csv(os.path.join(WORK_DIR, '10000', 'landmarks_10000.csv'), index=False)
landmarks_10000

,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,70,113,149,105,101,178,99,200,171,189
1,000002.jpg,68,110,148,110,96,160,74,195,154,195
2,000003.jpg,28,45,48,46,47,62,19,75,35,81
3,000004.jpg,174,282,362,282,308,430,140,499,293,499
4,000006.jpg,56,91,113,98,88,129,51,161,97,166
...,...,...,...,...,...,...,...,...,...,...,...
70285,202592.jpg,33,53,80,50,56,76,38,94,87,91
70286,202594.jpg,75,122,161,122,123,185,86,216,150,211
70287,202595.jpg,68,110,149,110,110,168,76,191,141,195
70288,202596.jpg,35,57,85,57,56,91,34,101,85,101


In [26]:
value_to_find = 1

# Считаем, сколько раз значение встречается в каждой колонке
counts = (select_10000 == value_to_find).sum()

# Оставляем только колонки, где счёт > 0, и сортируем по убыванию
counts_nonzero = counts.sort_values(ascending=True)

print("Количество вхождений по колонкам (по убыванию):")
print(counts_nonzero)

Количество вхождений по колонкам (по убыванию):
image_id                   0
Bald                    1004
Gray_Hair               1526
Wearing_Hat             2528
Double_Chin             2641
Chubby                  2706
Blurry                  2971
Eyeglasses              3072
Mustache                3635
Pale_Skin               4087
Wearing_Necktie         4260
Goatee                  4267
Sideburns               4495
Receding_Hairline       4582
Rosy_Cheeks             6521
Oval_Face               9015
5_o_Clock_Shadow        9699
Narrow_Eyes            10302
Bushy_Eyebrows         10917
Blond_Hair             12021
Bangs                  12804
Wearing_Necklace       13573
Brown_Hair             14290
Straight_Hair          15066
Black_Hair             16001
Wearing_Earrings       16105
Bags_Under_Eyes        19055
Big_Nose               19953
Pointy_Nose            22594
Male                   23385
Big_Lips               25982
Wavy_Hair              26269
Arched_Eyebrows        2